In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister, execute, Aer, IBMQ,BasicAer
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math
import matplotlib.pyplot as plt
from random import *

In [2]:
#permet de construire les entreés du circuit 
#et calcul le nombre de bit auxiliaire recquis

def buildInputs(numberOfQubits,valueA,valueB):
    
    stateVectorSize=int(math.pow(2,numberOfQubits))

    
    #Definition of the registers
    
    A = QuantumRegister(numberOfQubits,"a")
    circ_A=QuantumCircuit(A)
    B = QuantumRegister(numberOfQubits,"b")
    circ_B=QuantumCircuit(B)
    Cout = QuantumRegister(1,"cout")
    C = QuantumRegister(numberOfQubits,"c")
    circ_C=QuantumCircuit(C,Cout)
    
    #initializing circuit
    state_A = [0]*stateVectorSize #initialize to 0
    state_B = [0]*stateVectorSize #initialize to 0
    state_A[valueA-1]=1
    state_B[valueB-1]=1
    circ_A.initialize(state_A,A)
    circ_B.initialize(state_B,B)
    
    Adder = circ_A+circ_B+circ_C
    
    
    #compute g[i,i+1] into auxillary register for 0<i<n
    for i in range(0,numberOfQubits-1):
        Adder.ccx(A[i],B[i],C[i+1])
        
    Adder.ccx(A[numberOfQubits-1],B[numberOfQubits-1],Cout[0])
    
        
    Adder.barrier()    
    #compute p[i,i+1] into b register for 0<i<n
    for i in range(0,numberOfQubits):
        Adder.cx(A[i],B[i])
        
    
    Adder.barrier()
    

    for i in range(0,numberOfQubits-1):
        Adder.ccx(C[i],B[i],C[i+1])
    
    Adder.ccx(C[numberOfQubits-1],B[numberOfQubits-1],Cout[0])

    
    Adder.barrier()
    
    for i in range(0,numberOfQubits):
        Adder.cx(C[i],B[i])    
    
    Adder.barrier()
    
    
    for i in range(numberOfQubits-2,-1,-1):
        Adder.ccx(C[i],B[i],C[i+1])
        
    Adder.barrier()
    
    
    #compute g[i,i+1] into auxillary register for 0<i<n
    for i in range(numberOfQubits-2,-1,-1):
        Adder.ccx(A[i],B[i],C[i+1])
        
        
    #ajout d'un registre classique pour mesurer l'état des bits
    outB = ClassicalRegister(numberOfQubits,"outB")
    outC = ClassicalRegister(1,"outC")
    #ajout du circuit de mesure
    meas = QuantumCircuit(B,outB,Cout,outC)
    meas.barrier(Cout,B)
    meas.measure(B,outB)  #mesure la valeur de q et la place dans c
    meas.measure(Cout,outC)
    qc = Adder + meas   #ajoute le circuit de mesure à la suite de circ (le circuit avec les portes quantiques)
    
    return qc

In [3]:
def fromBinaryStringToInt(binString):
    n=binString[0]
    bitString=[]
    bitInt=0
    a=0
    for i in range(0,len(n)):
        if(n[i]!=" "):
            bitString.append(n[i])
    for i in range(len(bitString)-1,-1,-1):
        if(bitString[i]=="1"):
            bitInt=bitInt+int(math.pow(2,a))
        a=a+1
    return(bitInt)

In [12]:

#Test bench
def testBench():
    nbTest=0
    backendQASM = BasicAer.get_backend('qasm_simulator')
    for i in range(1,8):
        bound = math.pow(2,i)
        stateVectorSize=int(bound)
        for j in range(0,int(stateVectorSize/2)):
            valueA = randint(1,stateVectorSize)
            valueB = randint(1,stateVectorSize)
            Adder = buildInputs(i,valueA,valueB)
            job = execute(Adder,backendQASM,shots=1)
            result = job.result()
            counts = result.get_counts(Adder)
            nbTest=nbTest+1
            print(nbTest)
            if(valueA+valueB-2!=fromBinaryStringToInt(list(counts.keys()))):
                print("ERROR WRONG SUM !")
                print(valueA)
                print(valueB)
                print(i)
                return
    print("TEST COMPLETE")
        

In [13]:
testBench()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
TEST COMPLETE


In [18]:
A=buildInputs(15,5,3)
A.depth()

35

In [6]:
#Adder=buildInputs(20)
#Adder.size()